In [1]:
import pandas as pd
import numpy as np
import random

import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.51-b03, mixed mode)
  Starting server from C:\Users\Acer\anaconda3\envs\engine\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Acer\AppData\Local\Temp\tmpkvzgfiia
  JVM stdout: C:\Users\Acer\AppData\Local\Temp\tmpkvzgfiia\h2o_Acer_started_from_python.out
  JVM stderr: C:\Users\Acer\AppData\Local\Temp\tmpkvzgfiia\h2o_Acer_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_Acer_ldxch0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.501 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [2]:
tasks = pd.read_csv('data/30_2018/task_data.csv')
tasks_df = pd.read_csv('data/30_2018/task_train.csv')

target = 'TaskDelay'

X = tasks_df.copy()
y = X.pop(target)

In [3]:
project_ids = tasks['ProjectID'].unique().tolist()
indices = random.sample(project_ids, int(0.2 * len(project_ids)))
val_indices = random.sample(indices, int(0.5 * len(indices)))
test_indices = [x for x in indices if x not in val_indices]
val_idx = tasks.loc[tasks['ProjectID'].isin(val_indices)].index.tolist()
test_idx = tasks.loc[tasks['ProjectID'].isin(test_indices)].index.tolist()

val_frame = tasks_df[tasks_df.index.isin(val_idx)]
test_frame = tasks_df[tasks_df.index.isin(test_idx)]
train_frame = tasks_df[(~tasks_df.index.isin(test_idx)) & (~tasks_df.index.isin(val_idx))]

In [4]:
train_h2o = h2o.H2OFrame(train_frame)
val_h2o = h2o.H2OFrame(val_frame)
test_h2o = h2o.H2OFrame(test_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
x = train_h2o.columns
y = 'TaskDelay'  # replace with your actual response column
x.remove(y)

In [ ]:
aml = H2OAutoML(max_runtime_secs=1800,
                seed=42,
                stopping_metric='deviance', 
                nfolds=0)
aml.train(x=x, y=y, training_frame=train_h2o, validation_frame=val_h2o)

In [ ]:
lb = aml.leaderboard
print(lb)

In [ ]:
best = aml.leader
best.model_performance(test_h2o)

In [ ]:
h2o.save_model(model=best, path='model', force=True)

In [5]:
loaded_model = h2o.load_model("model/30_2018_30")

In [ ]:
loaded_model.model_performance(test_h2o)

In [6]:
preds = loaded_model.predict(test_h2o)
test_h2o['Preds'] = preds 
test_h2o.head(50)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


Cost,Priority,Progress,Duration,Worker,TaskLength,IsBadWeather,WeatherAssessment,StartDelay,DayCount,TaskDelay,Is_Delayed,Preds
2271,1,11.1111,9,5,0,1,37.5,0,0,1,0,1.34924
2271,1,22.2222,9,5,0,1,37.5,0,1,1,0,1.32734
2271,1,33.3333,9,5,0,1,37.5,0,2,1,0,1.3066
2271,1,33.3333,9,5,0,0,37.5,0,3,1,0,1.60341
2271,1,44.4444,9,5,0,0,37.5,0,4,1,0,1.52661
2271,1,55.5556,9,5,0,0,37.5,0,5,1,0,1.44549
2138,1,0,8,5,0,0,25,0,0,3,0,1.73432
2271,1,66.6667,9,5,0,0,37.5,0,6,1,0,1.33433
2138,1,12.5,8,5,0,0,25,0,1,3,0,1.65777
2271,1,77.7778,9,5,0,0,37.5,0,7,1,0,1.14891


In [22]:
preds.as_data_frame().iat[0, 0]

c:\Users\Acer\anaconda3\envs\engine\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


1.3492363306034336